<a href="https://colab.research.google.com/github/yusam/mead2/blob/main/autos_de_infracao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Carregando bibliotecas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
from google.colab import drive

### Montando drive do Google Drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


### Carregando dataset


In [3]:
infracao_aux_2018 = []
infracao_aux_2019 = []
infracao_aux_2020 = []


#le todos os arquivos de 2020
for doc in glob.glob("/content/drive/MyDrive/Colab Notebooks/autos_de_infracao/2020/*"):
    infracao_aux = pd.read_csv(doc,sep=";",encoding="latin-1",low_memory=False)
    infracao_aux.columns=["tipo_infracao",
                          "descricao",
                          "tipo_infrator",
                          "tipo_veiculo",
                          "data_cometimento",
                          "hora_cometimento",
                          "local_rodovia",
                          "local_km",
                          "local_referencia",
                          "local_complemento",
                          "latitude",
                          "longitude",
                          "gravidade"]
    infracao_aux_2020.append(infracao_aux)

    
   
#le todos os arquivos de 2019
for doc in glob.glob("/content/drive/MyDrive/Colab Notebooks/autos_de_infracao/2019/*"):
    infracao_aux = pd.read_csv(doc,sep=";",encoding="latin-1",low_memory=False)
    infracao_aux.columns=["tipo_infracao",
                          "descricao",
                          "tipo_infrator",
                          "tipo_veiculo",
                          "data_cometimento",
                          "hora_cometimento",
                          "local_rodovia",
                          "local_km",
                          "local_referencia",
                          "local_complemento",
                          "latitude",
                          "longitude",
                          "gravidade"]
    infracao_aux_2019.append(infracao_aux)


#le todos os arquivos de 2018
for doc in glob.glob("/content/drive/MyDrive/Colab Notebooks/autos_de_infracao/2018/*"):
    infracao_aux = pd.read_csv(doc,sep=";",encoding="latin-1",low_memory=False)
    if infracao_aux.shape[1] == 6:
        infracao_aux.columns=["tipo_infracao",
                              "descricao",
                              "tipo_infrator",
                              "tipo_veiculo",
                              "data_cometimento",
                              "hora_cometimento",]
    else:
        infracao_aux.columns=["tipo_infracao",
                              "descricao",
                              "tipo_infrator",
                              "tipo_veiculo",
                              "data_cometimento",
                              "hora_cometimento",
                              "local_rodovia",
                              "local_km",
                              "local_referencia",
                              "local_complemento",
                              "latitude",
                              "longitude",
                              "gravidade"]
    infracao_aux_2018.append(infracao_aux)    

In [4]:
## juntando dataframes de anos difrentes
infracao1 = pd.concat(infracao_aux_2018,ignore_index = True)
infracao2 = pd.concat(infracao_aux_2019,ignore_index = True)
infracao3 = pd.concat(infracao_aux_2020,ignore_index = True)

infracao = infracao1
infracao = infracao.append(infracao2)
infracao = infracao.append(infracao3)

del infracao1, infracao2, infracao3

infracao.head().T


,0,1,2,3,4
tipo_infracao,7455-0,7455-0,6769-0,7285-0,7579-0
descricao,Transitar em velocidade superior à máxima per...,Transitar em velocidade superior à máxima per...,Conduzir veíc c/ defeito no sist de iluminaçã...,Em movimento deixar de manter a placa traseir...,Recusar-se a ser submetido a teste exame clín...
tipo_infrator,Condutor,Condutor,Proprietário,Condutor,Condutor
tipo_veiculo,AUTOMOVEL,AUTOMOVEL,Automóvel,Automóvel,Automóvel
data_cometimento,01/07/2018,01/07/2018,01/07/2018,01/07/2018,01/07/2018
hora_cometimento,00:01,00:02,00:02,00:02,00:03
local_rodovia,NaN,NaN,NaN,NaN,NaN
local_km,NaN,NaN,NaN,NaN,NaN
local_referencia,NaN,NaN,NaN,NaN,NaN
local_complemento,NaN,NaN,NaN,NaN,NaN


In [5]:
## verificando tipo de variaveis
infracao.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2229315 entries, 0 to 899274
Data columns (total 13 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   tipo_infracao      object
 1   descricao          object
 2   tipo_infrator      object
 3   tipo_veiculo       object
 4   data_cometimento   object
 5   hora_cometimento   object
 6   local_rodovia      object
 7   local_km           object
 8   local_referencia   object
 9   local_complemento  object
 10  latitude           object
 11  longitude          object
 12  gravidade          object
dtypes: object(13)
memory usage: 238.1+ MB


In [6]:
## convertendo para data
infracao['data_cometimento'] = pd.to_datetime(infracao['data_cometimento'])
infracao.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2229315 entries, 0 to 899274
Data columns (total 13 columns):
 #   Column             Dtype         
---  ------             -----         
 0   tipo_infracao      object        
 1   descricao          object        
 2   tipo_infrator      object        
 3   tipo_veiculo       object        
 4   data_cometimento   datetime64[ns]
 5   hora_cometimento   object        
 6   local_rodovia      object        
 7   local_km           object        
 8   local_referencia   object        
 9   local_complemento  object        
 10  latitude           object        
 11  longitude          object        
 12  gravidade          object        
dtypes: datetime64[ns](1), object(12)
memory usage: 238.1+ MB


In [7]:
## verificando qtd de linhas e colunas
infracao.shape

(2229315, 13)

### Limpeza dos dados

#### Tratando valores duplicados

In [8]:
## verificando valores duplicados
infracao.duplicated().value_counts()

False    2154535
True       74780
dtype: int64

In [9]:
## dropando valores duplicados
infracao.drop_duplicates(inplace=True)
infracao.shape

(2154535, 13)

#### Tratando valores nulos

In [10]:
## verificando a quantidade de nulos
infracao.isna().sum().sort_values()

tipo_infracao              0
descricao                  0
tipo_infrator              0
data_cometimento           0
hora_cometimento           0
tipo_veiculo           11706
gravidade              89251
local_rodovia          89534
local_referencia     1672585
local_km             1685672
latitude             1697725
longitude            1697725
local_complemento    2017228
dtype: int64

In [11]:
## Convertendo campos para nulos - tipo de veiculo
infracao.loc[:,"tipo_veiculo"] = (infracao.loc[:,"tipo_veiculo"]                              
                                  .str.strip() .replace("-",np.nan)
                                  .replace("null",np.nan)
                                  .replace("INEXISTENTE",np.nan)                                                        
                                  .str.upper()
                                  )

In [12]:
## verificando a quantidade de nulos
infracao.isna().sum().sort_values()

tipo_infracao              0
descricao                  0
tipo_infrator              0
data_cometimento           0
hora_cometimento           0
tipo_veiculo           14894
gravidade              89251
local_rodovia          89534
local_referencia     1672585
local_km             1685672
latitude             1697725
longitude            1697725
local_complemento    2017228
dtype: int64

In [13]:
## dropando colunas nulas não utilizadas
infracao = infracao.drop(columns=['longitude','latitude','local_complemento','local_referencia','local_km','local_rodovia'])
infracao.isnull().sum().sort_values()

tipo_infracao           0
descricao               0
tipo_infrator           0
data_cometimento        0
hora_cometimento        0
tipo_veiculo        14894
gravidade           89251
dtype: int64

In [14]:
## tratando valores nulos
## utilizando a moda para os tipos de veiculos 
infracao['tipo_veiculo'].fillna(value = infracao['tipo_veiculo'].mode().to_string(), inplace=True)
infracao.isnull().sum().sort_values()

tipo_infracao           0
descricao               0
tipo_infrator           0
tipo_veiculo            0
data_cometimento        0
hora_cometimento        0
gravidade           89251
dtype: int64

#### Normalizando dados

In [15]:
## padronizando tipo infracao
infracao.loc[:,"tipo_infracao"] = infracao.loc[:,"tipo_infracao"].str.strip()
infracao.tipo_infracao.value_counts()

7455-0    1200832
5681-0     282471
7463-0     109974
6769-0      77162
7633-2      45663
           ...   
6394-1          1
6009-5          1
5088-2          1
6297-0          1
5100-1          1
Name: tipo_infracao, Length: 295, dtype: int64

In [16]:
## padronizando tipo infrator
infracao.loc[:,"tipo_infrator"] = (infracao.loc[:,"tipo_infrator"]
                                   .str.strip()    
                                   .str.replace("(Proprietário|ProprietÃ¡rio)","PROPRIETARIO")
                                   .str.replace("(Pessoa Jurídica|Pessoa JurÃ\xaddica)","PESSOA_JURIDICA")
                                   .str.replace("(Pessoa Física|Pessoa FÃ\xadsica)","PESSOA_FISICA")
                                   .str.replace("Servidor Público","SERVIDOR_PUBLICO")                               
                                   .str.upper()
                                   )
infracao.tipo_infrator.value_counts()

CONDUTOR            2038019
PROPRIETARIO         116470
EMB/TRANSP               13
PESSOA_JURIDICA          11
PESSOA_FISICA            11
SERVIDOR_PUBLICO         11
Name: tipo_infrator, dtype: int64

In [17]:
## padronizando gravidade da infracao
infracao.loc[:,"gravidade"] = (infracao.loc[:,"gravidade"]
                               .str.strip()
                               .str.replace("(MÃ©dia)","MEDIA")
                               .str.replace("GravÃ\xadssima","GRAVISSIMA")                           
                               .str.upper()
                               )
infracao.gravidade.value_counts()

MEDIA         1294554
LEVE           298883
GRAVE          251225
GRAVISSIMA     220622
Name: gravidade, dtype: int64

In [18]:
## padronizando nome tipo de veiculo
infracao.loc[:,"tipo_veiculo"] = (infracao.loc[:,"tipo_veiculo"]    
                                  .str.replace("(CARRO-DE).*","CARRO_DE_MAO") 
                                  .str.replace("(AUTOM|0    AUTO).*","AUTOMOVEL")   
                                  .str.replace("(MOTOCICLET).*","MOTOCICLETA")
                                  .str.replace("(CAMINHONET).*","CAMINHONETE")
                                  .str.replace("(CAMINH)[^o|^O].[^\s]*|C[\.]","CAMINHAO")
                                  .str.replace("(\sTRATOR|\sTrator)","_TRATOR")
                                  .str.replace("(UTILI).*","UTILITARIO")
                                  .str.replace("(TRATOR).*","TRATOR_DE_RODAS")
                                  .str.replace("(.NIBU|Ã\x94).*","ONIBUS")
                                  .str.replace("(MICR|Ã\x94).*","MICROONIBUS")
                                  .str.replace("[TRIC](ICLOM).*","CICLOMOTOR")
                                  .str.replace("(SEMI|S\.)(-|)","SEMI")
                                  .str.replace("(MOTOR CASA|MOTOR-CASA)","TRAILER")
                                  .str.replace("CHASSI PLATAFORMA","CHASSI_PLATAFORMA")
                                  .str.replace("TR RODAS","TRANSP_RODAS")
                                  .str.replace("TR MISTO","TRANSP_MISTO")
                                  .str.replace("TR ESTEIRAS","TRANSP_ESTEIRAS")  
                                  )
infracao.tipo_veiculo.value_counts()

AUTOMOVEL                   1499607
CAMINHONETE                  204195
CAMIONETA                    135765
MOTOCICLETA                  107504
CAMINHAO                      72489
UTILITARIO                    70589
CAMINHAO_TRATOR_DE_RODAS      20160
SEMIREBOQUE                   17793
MICROONIBUS                    8496
ONIBUS                         7629
MOTONETA                       6602
REBOQUE                        3323
TRICICLO                        141
CICLOMOTOR                      136
TRAILER                          39
TRANSP_RODAS                     15
CHASSI_PLATAFORMA                11
SIDE-CAR                         10
TRANSP_MISTO                      8
BICICLETA                         6
CHARRETE                          5
CARRO_DE_MAO                      4
TRANSP_ESTEIRAS                   3
BONDE                             2
QUADRICICLO                       1
TRATOR_DE_RODAS                   1
                                  1
Name: tipo_veiculo, dtype: i

#### tratando campo gravidade

In [19]:
infracao.gravidade.value_counts(normalize=True)

MEDIA         0.626816
LEVE          0.144718
GRAVE         0.121642
GRAVISSIMA    0.106824
Name: gravidade, dtype: float64

In [20]:
infracao.isnull().sum().sort_values()

tipo_infracao           0
descricao               0
tipo_infrator           0
tipo_veiculo            0
data_cometimento        0
hora_cometimento        0
gravidade           89251
dtype: int64

In [21]:
## tratando valores nulos
## utilizando a moda para os tipos de veiculos 
infracao['gravidade'].fillna(value = infracao['gravidade'].mode().to_string(), inplace=True)
infracao.isnull().sum().sort_values()

tipo_infracao       0
descricao           0
tipo_infrator       0
tipo_veiculo        0
data_cometimento    0
hora_cometimento    0
gravidade           0
dtype: int64

In [22]:
## padronizando gravidade da infracao
infracao.loc[:,"gravidade"] = (infracao.loc[:,"gravidade"]
                               .str.strip()
                               .str.replace("(MÃ©dia|0    MEDIA)","MEDIA")
                               .str.replace("GravÃ\xadssima","GRAVISSIMA")                           
                               .str.upper()
                               )
infracao.gravidade.value_counts(normalize=True)

MEDIA         0.642275
LEVE          0.138723
GRAVE         0.116603
GRAVISSIMA    0.102399
Name: gravidade, dtype: float64

### Gerando arquivo tratado

In [24]:
## exportando resultado
infracao.to_csv('/content/drive/MyDrive/Colab Notebooks/autos_de_infracao/infracoes_2018_2020_tratado.csv', sep=';',encoding="latin-1",index=False)